In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../")
from config import weather_api_key
from citipy import citipy
import requests

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = list(zip(lats, lngs))

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

748

In [4]:
url = f"http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID={weather_api_key}"

In [5]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | port alfred
Processing Record 2 of Set 1 | cape town
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | pangkalanbuun
Processing Record 5 of Set 1 | sarangani
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | rio branco
Processing Record 8 of Set 1 | hilo
Processing Record 9 of Set 1 | kloulklubed
Processing Record 10 of Set 1 | dunedin
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | sivas
Processing Record 13 of Set 1 | kapaa
Processing Record 14 of Set 1 | yaan
Processing Record 15 of Set 1 | severo-kurilsk
Processing Record 16 of Set 1 | sioux lookout
Processing Record 17 of Set 1 | xinyang
Processing Record 18 of Set 1 | carnarvon
Processing Record 19 of Set 1 | saldanha
Processing Record 20 of Set 1 | beringovskiy
Processing Record 21 of Set 1 | koumac
Processing Record 22 of Set 1 | nikolskoye
Processing Record 23 of Set 1 |

In [6]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Alfred,ZA,-33.59,26.89,65.34,84,23,12.86,light rain
1,Cape Town,ZA,-33.93,18.42,66.00,82,20,21.92,few clouds
2,Albany,US,42.60,-73.97,48.99,65,56,1.01,broken clouds
3,Pangkalanbuun,ID,-2.68,111.62,74.84,97,100,2.89,overcast clouds
4,Sarangani,PH,5.40,125.46,82.71,74,100,8.21,light rain
5,Punta Arenas,CL,-53.15,-70.92,48.20,57,75,25.28,broken clouds
6,Rio Branco,BR,-9.97,-67.81,77.00,94,40,8.05,shower rain
7,Hilo,US,19.73,-155.09,73.99,68,1,6.93,clear sky
8,Kloulklubed,PW,7.04,134.26,83.19,76,92,10.45,overcast clouds
9,Dunedin,NZ,-45.87,170.50,53.01,77,32,23.00,scattered clouds


In [7]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")